In [2]:
import glob
import os
import pandas as pd
import re
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import date, timedelta, datetime
import sklearn.preprocessing as preproc
import sklearn.linear_model as lm
import sklearn.pipeline
from sklearn import decomposition
from scipy.optimize import least_squares
import geopandas as gpd
from requests import get,Session
import wget
from census import Census
from us import states
import folium
%matplotlib inline

In [3]:
# read in 2016 census tract shapefile (from https://www.census.gov/geo/maps-data/data/cbf/cbf_tracts.html)
gdf = gpd.read_file('/Users/nehrkorn/Downloads/cb_2016_06_tract_500k/cb_2016_06_tract_500k.shp')

Variable code | Census description 
:---|:---
DP05_0082E | Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population
DP03_0032E | Estimate!!INDUSTRY!!Civilian employed population 16 years and over 
DP03_0033PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining 
DP03_0034PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Construction 
DP03_0035PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Manufacturing 
DP03_0036PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Wholesale trade 
DP03_0037PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Retail trade 
DP03_0038PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Transportation and warehousing, and utilities 
DP03_0039PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Information 
DP03_0040PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Finance and insurance, and real estate and rental and leasing 
DP03_0041PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Professional, scientific, and management, and administrative and waste management services 
DP03_0042PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Educational services, and health care and social assistance 
DP03_0043PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Arts, entertainment, and recreation, and accommodation and food services 
DP03_0044PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Other services, except public administration 
DP03_0045PE | Percent!!INDUSTRY!!Civilian employed population 16 years and over!!Public administration 
DP03_0096PE | Percent!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage 
DP03_0099PE | Percent!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!No health insurance coverage 
DP03_0062E | Estimate!!INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars) 
DP03_0063E | Estimate!!INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Total households!!Mean household income (dollars) 
DP05_0017E | Estimate!!SEX AND AGE!!Median age (years) 
DP05_0018PE | Percent!!SEX AND AGE!!18 years and over 
DP05_0020PE | Percent!!SEX AND AGE!!62 years and over 
DP03_0119PE | Estimate!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families 

In [51]:
census_vars_dict = {'DP05_0082E':'dp_pop_voting_age',
                    'DP03_0032E':'dp_pop_employed',
                    'DP03_0033PE':'dp_pct_industry_farming',
                    'DP03_0034PE':'dp_pct_industry_construction',
                    'DP03_0035PE':'dp_pct_industry_manufacturing',
                    'DP03_0036PE':'dp_pct_industry_wholesale',
                    'DP03_0037PE':'dp_pct_industry_retail',
                    'DP03_0038PE':'dp_pct_industry_transportation',
                    'DP03_0039PE':'dp_pct_industry_information',
                    'DP03_0040PE':'dp_pct_industry_finance',
                    'DP03_0041PE':'dp_pct_industry_professional',
                    'DP03_0042PE':'dp_pct_industry_education',
                    'DP03_0043PE':'dp_pct_industry_arts',
                    'DP03_0044PE':'dp_pct_industry_services',
                    'DP03_0045PE':'dp_pct_industry_publicadmin',
                    'DP03_0096PE':'dp_pct_healthinsurance_yes',
                    'DP03_0099PE':'dp_pct_healthinsurance_no',
                    'DP03_0062E':'dp_median_household_income',
                    'DP03_0063E':'dp_mean_household_income',
                    'DP05_0017E':'dp_median_age',
                    'DP05_0018PE':'dp_pct_age_18plus',
                    'DP05_0020PE':'dp_pct_age_62plus',
                    'DP03_0119PE':'dp_pct_familiesbelowFPL'}

In [52]:
# should return tract name and population for all tracts in CA
c = Census("fc40d0e5624aa1c5f10cb80e68c8a62d1d5bdc68")
result = c.acs5dp.get(['NAME']+list(census_vars_dict.keys()),
                      geo={'for': 'tract:*',
                           'in': 'state:{}'.format(states.CA.fips)})

[{'DP03_0032E': 1568.0,
  'DP03_0033PE': 0.6,
  'DP03_0034PE': 2.2,
  'DP03_0035PE': 5.9,
  'DP03_0036PE': 2.6,
  'DP03_0037PE': 10.5,
  'DP03_0038PE': 4.6,
  'DP03_0039PE': 3.8,
  'DP03_0040PE': 14.8,
  'DP03_0041PE': 24.9,
  'DP03_0042PE': 24.6,
  'DP03_0043PE': 1.6,
  'DP03_0044PE': 2.1,
  'DP03_0045PE': 1.9,
  'DP03_0062E': 177417.0,
  'DP03_0063E': 247448.0,
  'DP03_0096PE': 96.4,
  'DP03_0099PE': 3.6,
  'DP03_0119PE': 3.0,
  'DP05_0017E': 54.2,
  'DP05_0018PE': 87.2,
  'DP05_0020PE': 31.5,
  'DP05_0082E': 2423.0,
  'NAME': 'Census Tract 4001, Alameda County, California',
  'county': '001',
  'state': '06',
  'tract': '400100'},
 {'DP03_0032E': 1229.0,
  'DP03_0033PE': 0.5,
  'DP03_0034PE': 4.5,
  'DP03_0035PE': 4.5,
  'DP03_0036PE': 0.0,
  'DP03_0037PE': 5.0,
  'DP03_0038PE': 1.9,
  'DP03_0039PE': 8.7,
  'DP03_0040PE': 9.5,
  'DP03_0041PE': 24.1,
  'DP03_0042PE': 26.5,
  'DP03_0043PE': 5.9,
  'DP03_0044PE': 7.2,
  'DP03_0045PE': 1.8,
  'DP03_0062E': 153125.0,
  'DP03_0063E': 1884

In [55]:
# make dataframe out of list of dicts
df = pd.DataFrame(result)
# add a tract code column that matches the shapefile (for merging)
df['GEOID'] = df.state+df.county+df.tract
df.rename(index=str,columns=census_vars_dict,inplace=True)
df

,dp_pop_employed,dp_pct_industry_farming,dp_pct_industry_construction,dp_pct_industry_manufacturing,dp_pct_industry_wholesale,dp_pct_industry_retail,dp_pct_industry_transportation,dp_pct_industry_information,dp_pct_industry_finance,dp_pct_industry_professional,...,dp_pct_familiesbelowFPL,dp_median_age,dp_pct_age_18plus,dp_pct_age_62plus,dp_pop_voting_age,NAME,county,state,tract,GEOID
0,1568.0,0.6,2.2,5.9,2.6,10.5,4.6,3.8,14.8,24.9,...,3.0,54.2,87.2,31.5,2423.0,"Census Tract 4001, Alameda County, California",001,06,400100,06001400100
1,1229.0,0.5,4.5,4.5,0.0,5.0,1.9,8.7,9.5,24.1,...,1.1,43.6,82.9,24.9,1537.0,"Census Tract 4002, Alameda County, California",001,06,400200,06001400200
2,3198.0,0.8,2.1,6.2,0.9,4.8,2.5,2.8,12.3,25.9,...,3.6,41.3,82.4,15.2,3997.0,"Census Tract 4003, Alameda County, California",001,06,400300,06001400300
3,2564.0,0.5,2.7,1.1,3.9,4.0,3.0,5.7,4.3,20.8,...,0.9,36.6,84.3,14.7,3326.0,"Census Tract 4004, Alameda County, California",001,06,400400,06001400400
4,2463.0,0.8,5.9,3.0,4.9,8.2,4.9,4.7,5.1,18.3,...,5.8,34.1,86.7,12.3,2850.0,"Census Tract 4005, Alameda County, California",001,06,400500,06001400500
5,1012.0,0.0,5.8,6.0,1.0,10.1,1.8,4.8,4.7,12.5,...,3.3,34.8,86.0,12.7,1366.0,"Census Tract 4006, Alameda County, California",001,06,400600,06001400600
6,2485.0,0.0,4.4,4.1,0.9,13.9,3.7,2.4,2.3,15.7,...,12.3,35.9,83.7,15.1,3458.0,"Census Tract 4007, Alameda County, California",001,06,400700,06001400700
7,2087.0,0.9,7.1,4.6,0.0,7.1,5.7,3.5,7.6,18.4,...,7.4,37.0,88.2,16.7,2839.0,"Census Tract 4008, Alameda County, California",001,06,400800,06001400800
8,1373.0,0.0,5.0,5.7,2.1,9.0,2.0,3.6,6.4,27.5,...,10.5,36.1,81.9,12.8,1650.0,"Census Tract 4009, Alameda County, California",001,06,400900,06001400900
9,3621.0,0.0,5.3,5.1,5.5,6.7,4.7,3.1,6.8,18.2,...,19.9,36.0,86.8,9.7,4896.0,"Census Tract 4010, Alameda County, California",001,06,401000,06001401000
